In [1]:
%autosave 0
# %matplotlib inline
%matplotlib notebook

from __future__ import division, print_function
import numpy as np
from tensorflow.python.client import device_lib
from keras import backend as K
import tensorflow as tf
import keras
from keras.datasets import cifar10


print(device_lib.list_local_devices())
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(keras.__version__)

Autosave disabled
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5654338391016922748
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2944794624
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 16977938223014435259
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:06:00.0, compute capability: 3.7"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 172490752
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 15674934906972793512
physical_device_desc: "device: 1, name: Tesla K80, pci bus id: 0000:07:00.0, compute capability: 3.7"
]
2.1.6


Using TensorFlow backend.


In [4]:
from keras.layers import Input, Dense, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Add, Activation, Conv2DTranspose, Reshape
from keras.models import Model, load_model
from keras import optimizers
# from keras.utils import multi_gpu_model

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
input_train_shape = x_train.shape[1:]

n_out = 10
n_fc1 = 256

rc_model = load_model('cifar_recurrentsimple_180713.h5');
conv1_ffw = rc_model.get_layer('conv_ff1')
conv1_rcw = rc_model.get_layer('conv_rc1')
conv2_ffw = rc_model.get_layer('conv_ff22')
conv2_rcw = rc_model.get_layer('conv_rc2')
conv3_ffw = rc_model.get_layer('conv_ff32')
conv3_rcw = rc_model.get_layer('conv_rc3')
conv4_ffw = rc_model.get_layer('conv_ff42')
conv4_rcw = rc_model.get_layer('conv_rc4')
fc5_ffw = rc_model.get_layer('fc_ff52')
fc5_rcw = rc_model.get_layer('fc_rc5')
fc6_ffw = rc_model.get_layer('fc_ff62')
fc6_rcw = rc_model.get_layer('fc_rc6')
fc7_ffw = rc_model.get_layer('fc_ff72')
fc7_rcw = rc_model.get_layer('fc_rc7')

input_layer = Input(shape=(input_train_shape), name='input')

#First feedforward
conv1_ff1 = Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_ff11', trainable=False, weights=conv1_ffw.get_weights())(input_layer)
bn1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_11')(conv1_ff1)
pool1 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool_11')(bn1)
conv2_ff1 = Conv2D(128, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_ff21', trainable=False, weights=conv2_ffw.get_weights())(pool1)
bn2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_21')(conv2_ff1)
pool2 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool_2')(bn2)
conv3_ff1 = Conv2D(256, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_ff31', trainable=False, weights=conv3_ffw.get_weights())(pool2)
bn3 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_31')(conv3_ff1)
pool3 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool31')(bn3)
conv4_ff1 = Conv2D(1024, (4,4), strides=(1,1), padding='valid', activation='relu', name='conv_ff41', trainable=False, weights=conv4_ffw.get_weights())(pool3)
bn4 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_41')(conv4_ff1)
flat1 = Flatten(name='flat_11')(bn4)
fc5_ff1 = Dense(1024, activation='relu', name='fc_ff51', trainable=False, weights=fc5_ffw.get_weights())(flat1)
bn5 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_51')(fc5_ff1)
fc6_ff1 = Dense(256, activation='relu', name='fc_ff61', trainable=False, weights=fc6_ffw.get_weights())(bn5)
bn6 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_61')(fc6_ff1)
fc7_ff1 = Dense(10, activation='linear', name='fc_ff71', trainable=False, weights=fc7_ffw.get_weights())(bn6)


#Layer 1 complex
conv1_rc1 = Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_rc11', trainable=False, weights=conv1_rcw.get_weights())(conv1_ff1)
deconv1_td1 = Conv2DTranspose(64, (2,2), strides=(2,2), padding='valid', activation='relu', name='deconv_td1', trainable=True)(conv2_ff1)
conv1_mrg1 = Add()([conv1_ff1, conv1_rc1, deconv1_td1])
bn12 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_12')(conv1_mrg1)
pool12 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool_12')(bn12)

#Layer 2 complex
conv2_ff2 = Conv2D(128, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_ff22', trainable=False, weights=conv2_ffw.get_weights())(pool12)
conv2_rc1 = Conv2D(128, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_rc21', trainable=False, weights=conv2_rcw.get_weights())(conv2_ff1)
deconv2_td1 = Conv2DTranspose(128, (2,2), strides=(2,2), padding='valid', activation='relu', name='deconv_td2', trainable=True)(conv3_ff1)
conv2_mrg1 = Add()([conv2_ff2, conv2_rc1, deconv2_td1])
bn22 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_22')(conv2_mrg1)
pool22 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool_22')(bn22)


# Layer 3 complex
conv3_ff2 = Conv2D(256, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_ff32', trainable=False, weights=conv3_ffw.get_weights())(pool22)
conv3_rc1 = Conv2D(256, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_rc31', trainable=False, weights=conv3_rcw.get_weights())(conv3_ff1)
deconv3_td1 = Conv2DTranspose(256, (8,8), strides=(1,1), padding='valid', activation='relu', name='deconv_td3', trainable=True)(conv4_ff1)
conv3_mrg1 = Add()([conv3_ff2, conv3_rc1, deconv3_td1])
bn32 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_32')(conv3_mrg1)
pool32 = MaxPooling2D(pool_size=(2,2), padding='valid', name='pool_32')(bn32)

# Layer 4 complex
conv4_ff2 = Conv2D(1024, (4,4), strides=(1,1), padding='valid', activation='relu', name='conv_ff42', trainable=False, weights=conv4_ffw.get_weights())(pool32)
conv4_rc1 = Conv2D(1024, (1,1), strides=(1,1), padding='valid', activation='relu', name='conv_rc41', trainable=False, weights=conv4_rcw.get_weights())(conv4_ff1)
deconv4_td1 = Dense(1024, activation='relu', name='deconv_td4', trainable=True)(fc5_ff1)
resh11 = Reshape((1,1,1024))(deconv4_td1)
conv4_mrg1 = Add()([conv4_ff2, conv4_rc1, resh11])
bn42 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_42')(conv4_mrg1)
flat12 = Flatten(name='flat_12')(bn42)


# Layer 5 complex
fc5_ff2 = Dense(1024, activation='relu', name='fc_ff52', trainable=False, weights=fc5_ffw.get_weights())(flat12)
fc5_rc1 = Dense(1024, activation='relu', name='fc_rc51', trainable=False, weights=fc5_rcw.get_weights())(fc5_ff1)
fc5_td1 = Dense(1024, activation='relu', name='fc_td5', trainable=True)(fc6_ff1)
fc5_mrg1 = Add()([fc5_ff2, fc5_rc1, fc5_td1])
bn52 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_52')(fc5_mrg1)

# Layer 6 complex
fc6_ff2 = Dense(256, activation='relu', name='fc_ff62', trainable=False, weights=fc6_ffw.get_weights())(bn52)
fc6_rc1 = Dense(256, activation='relu', name='fc_rc61', trainable=False, weights=fc6_rcw.get_weights())(fc6_ff1)
fc6_td1 = Dense(256, activation='relu', name='fc_td6', trainable=True)(fc7_ff1)
fc6_mrg1 = Add()([fc6_ff2, fc6_rc1, fc6_td1])
bn62 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name='bn_62')(fc6_mrg1)

# Layer 7 complex
fc7_ff2 = Dense(10, activation='linear', name='fc_ff72', trainable=False, weights=fc7_ffw.get_weights())(bn62)
fc7_rc1 = Dense(10, activation='linear', name='fc_rc71', trainable=False, weights=fc7_rcw.get_weights())(fc7_ff1)
fc7_mrg1 = Add()([fc7_ff2, fc7_rc1])
output_layer = Activation('softmax')(fc7_mrg1)




In [5]:
sgd = optimizers.SGD(lr=2, decay=1e-6, momentum=0.9)
model = Model(input_layer, output_layer)
# pmodel = multi_gpu_model(model)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv_ff11 (Conv2D)              (None, 32, 32, 64)   1792        input[0][0]                      
__________________________________________________________________________________________________
bn_11 (BatchNormalization)      (None, 32, 32, 64)   256         conv_ff11[0][0]                  
__________________________________________________________________________________________________
pool_11 (MaxPooling2D)          (None, 16, 16, 64)   0           bn_11[0][0]                      
__________________________________________________________________________________________________
conv_ff21 

In [6]:
print (x_train.shape)
print (y_train.shape)

# scale intensities between 0 and 1
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print('min:', x_train[0].min())
print('max:', x_train[0].max())


(50000, 32, 32, 3)
(50000, 1)
min: 0.0
max: 1.0


In [7]:
n_epochs = 5
n_batch_size = 4
labels_train = keras.utils.to_categorical(y_train, n_out)
labels_test = keras.utils.to_categorical(y_test, n_out)


history = model.fit(x_train, labels_train,
                    epochs=n_epochs,
                    batch_size=n_batch_size)

evaluation = model.evaluate(x_test, labels_test, verbose=0)

print('\n[Train parameters]')
for item in history.params:
    print(item+':', history.params[item])
    
print('\n[Model evaluation]')
print('test', history.params['metrics'][0], format(np.mean(evaluation[0]), '.4f'))
print('test', history.params['metrics'][1], format(np.mean(evaluation[1]), '.4f'))

Epoch 1/5
50000/50000 [==============================] - 1046s 21ms/step - loss: 0.9871 - acc: 0.6694
Epoch 2/5
50000/50000 [==============================] - 866s 17ms/step - loss: 0.9065 - acc: 0.6958
Epoch 3/5
50000/50000 [==============================] - 876s 18ms/step - loss: 0.8717 - acc: 0.7075
Epoch 4/5
50000/50000 [==============================] - 871s 17ms/step - loss: 0.8493 - acc: 0.7128
Epoch 5/5
50000/50000 [==============================] - 896s 18ms/step - loss: 0.8333 - acc: 0.7202

[Train parameters]
metrics: ['loss', 'acc']
samples: 50000
batch_size: 4
epochs: 5
steps: None
do_validation: False
verbose: 1

[Model evaluation]
test loss 1.0669
test acc 0.6618


In [8]:
model.save('cifar_rectopdownsimple_180717.h5')

In [ ]:
# Playground
conv1_ffw.set_weights(1)